# 프롬프트(시스템 역할 메시지)를 통한 챗봇 구현

## OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# .env 로드 및 API 키 확인
load_dotenv()

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

## 프롬프트 - 시스템 역할 메시지 및 사용자 메시지 구성
- `system` 역할: LLM에게 '명탐정 코난 전문가' 역할 부여
- `human` 역할: 사용자 질문 삽입

In [ ]:
# 초기 시스템 메시지
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 '명탐정 코난' 전문가입니다. 모든 질문에 친절하고 정확하게 답해주세요."),
    ("human", "{question}")
])

## UI와 LLM 입출력 처리 함수
Gradio UI의 입력을 LLM으로 전달하고, LLM의 결과를 UI 출력으로 연동하는 인터페이스 함수

In [ ]:
def conan_expert_reply(user_input):
    # 사용자 입력(user_input)을 이용해 프롬프트 메시지 구성
    prompt_messages = prompt.format_messages(question=user_input)

    # LLM으로 프롬프트 메시지 전달하여 응답 수신
    ai_message = llm.invoke(prompt_messages)
    print(ai_message)

    # LLM 전달 메시지 파서
    parser = StrOutputParser()
    
    # UI로 출력할 content 부분만 전달
    return parser.invoke(ai_message)

## Gradio 인터페이스 구성

패키지 설치 `pip install gradio`

Gradio의 Blocks UI 구성 방식을 사용해 명탐정 코난 퀴즈 챗봇 인터페이스를 구성
- `with gr.Blocks() as demo `: Gradio의 레이아웃 기반 UI 블록을 시작 - 앱 전체 인스턴스를 `demo`로 선언
- `gr.Markdown(...)` : 화면 상단에 표시할 설명 문구 (Markdown 문법 사용)
- `gr.Textbox(...)` : 사용자가 입력할 수 있는 텍스트 입력 또는 출력창 (placeholder로 입력 힌트를 표시 가능)
- `demo.launch()` : Gradio UI 실행

※ 입력 처리 상세 설명
- `question.submit(fn=conan_expert_reply, inputs=question, outputs=answer)`
    - `question.submit(...)` : 사용자가 텍스트박스에 입력하고 Enter를 누르면 실행
    - `conan_expert_reply` : 사용자의 입력을 받아 LLM에 전달하고, LLM의 응답 메시지를 UI로 전달 
    - `inputs=question` : 사용자가 입력한 질문 메시지
    - `outputs=answer` : 사용자의 질문에 대한 LLM의 응답 메시지


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️‍♂️ 명탐정 코난 전문가 (질의 & 응답)")

    # 입/출력 영역 구성
    question = gr.Textbox(label="질문을 입력하세요", placeholder="코난이 작아진 이유는 뭐죠?")
    answer = gr.Textbox(label="코난 전문가의 답변", lines=5)

    # 입력 이벤트 처리
    question.submit(fn=conan_expert_reply, inputs=question, outputs=answer)

# Gradio 실행
demo.launch()

-----
** End of Documents **